<a href="https://colab.research.google.com/github/anvy87/AlgoTrading/blob/master/nsepy_bhavcopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install nsepy

  Created wheel for nsepy: filename=nsepy-0.8-cp36-none-any.whl size=36084 sha256=c8a0841380feaba23e75f97769b399cb86da1b2276024c5e53a395c356c02e70
  Stored in directory: /root/.cache/pip/wheels/05/2c/72/5324e267426eea65dec0c21a12a48b15887c0cf686c7a8fe11
Successfully built nsepy


In [2]:
from nsepy import get_history
from datetime import date

In [3]:
from nsepy.history import get_price_list

In [28]:
def get_delivery_position(dt, segment='EQ'):
    MMM = dt.strftime("%b").upper()
    yyyy = dt.strftime("%Y")

    """
    1. ddmmyyyy
    """
    res = daily_deliverypositions_url(dt.strftime("%d%m%Y").upper())
    text = res.content.decode()
    fp = six.StringIO(text)

    # The file starts with initial lines that just have text infomation
    # e.g.
    # Security Wise Delivery Position - Compulsory Rolling Settlement
    # 10,MTO,19072019,471778636,0001790
    # Trade Date <19-JUL-2019>,Settlement Type <N>

    # Skip the initial lines till we get to the actual data

    df = pd.read_csv(fp, names=["RECORD TYPE", "SR NO", "SYMBOL", "SEGMENT", "TRADE VOLUME", "TOTDELQTY", "PCT DEL TO TRADE"],
                     header=None, skiprows=4,
                     usecols=["SYMBOL", "SEGMENT", "TRADE VOLUME",
                              "TOTDELQTY", "PCT DEL TO TRADE"]
                     )
    flsegment = df['SEGMENT'] == segment
    df = df[flsegment]

    return df

In [17]:
from nsepy.commons import URLFetch
from requests import Session
from functools import partial
from nsepy.constants import symbol_count, symbol_list

In [18]:
session = Session()
# headers = {
    # 'Host': 'www1.nseindia.com',
    # 'Referer': 'https://www1.nseindia.com/products/content/equities/equities/eq_security.htm'}

headers = {'Accept': '*/*',
           'Accept-Encoding': 'gzip, deflate, sdch, br',
           'Accept-Language': 'en-GB,en-US;q=0.8,en;q=0.6',
           'Connection': 'keep-alive',
           'Host': 'www1.nseindia.com',
           'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
           'X-Requested-With': 'XMLHttpRequest'}

In [23]:
URLFetchSession = partial(URLFetch, session=session, headers=headers)

In [29]:
daily_deliverypositions_url = URLFetchSession(url='https://www1.nseindia.com/archives/equities/mto/MTO_%s.DAT')


In [30]:
dlvry = get_delivery_position(dt=date(2021,1,1))

NameError: ignored

In [27]:
dlvry

In [11]:
prices = get_price_list(dt=date(2021,1,1))

In [5]:
prices.to_csv('bhavcopy.csv')

In [12]:
prices

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN
0,20MICRONS,EQ,40.75,40.80,39.80,40.05,40.00,40.20,126229,5.068836e+06,01-JAN-2021,803,INE144J01027
1,21STCENMGM,EQ,10.65,10.65,10.45,10.65,10.65,10.45,978,1.038775e+04,01-JAN-2021,15,INE253B01015
2,3IINFOTECH,EQ,6.75,6.75,6.75,6.75,6.75,6.45,666689,4.500151e+06,01-JAN-2021,467,INE748C01020
3,3MINDIA,EQ,20830.70,21243.95,20830.70,21127.60,21098.00,20830.55,1663,3.499062e+07,01-JAN-2021,1079,INE470A01017
4,3PLAND,EQ,11.35,11.65,10.80,11.20,11.05,11.35,18637,2.058212e+05,01-JAN-2021,88,INE105C01023
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003,ZODJRDMKJ,EQ,30.00,30.00,28.70,29.05,29.05,29.00,2816,8.227275e+04,01-JAN-2021,113,INE077B01018
2004,ZOTA,EQ,179.35,184.90,176.20,178.05,178.50,178.50,76262,1.370110e+07,01-JAN-2021,1048,INE358U01012
2005,ZUARI,EQ,99.80,100.00,97.00,97.85,98.50,99.20,85627,8.464155e+06,01-JAN-2021,1511,INE840M01016
2006,ZUARIGLOB,EQ,72.95,74.55,71.00,71.30,71.35,72.30,64075,4.609581e+06,01-JAN-2021,1046,INE217A01012
